# Задание 3
Создать процедуру расчета рабочих дней.

Входящие параметры:

Дата начала периода расчета

Дата окончания периода расчета.

Ожидаемый результать выполнения хранимой процедуры - заполнение таблицы T_CONTRACTOR_WORK_DAY рабочими днями согласно расписания работы поставщиков из таблицы T_CONTRACTOR_SHERULER Выходные дни (В) не должны попадать в таблицу T_CONTRACTOR_WORK_DAY

Пример выполнения для Поставщика 1 (Из примера записи таблицы T_CONTRACTOR_SHERULER) с параметрами '01.01.2019' - '08.01.2019' таблица T_CONTRACTOR_WORK_DAY заполнится следующими данными:

- 1 Поставщик 1 01.01.2019 08:00 01.01.2019 20:00
- 2 Поставщик 1 03.01.2019 08:00 04.01.2019 08:00
- 3 Поставщик 1 04.01.2019 08:00 04.01.2019 08:00
- 4 Поставщик 1 05.01.2019 20:00 06.01.2019 08:00
- 5 Поставщик 1 07.01.2019 08:00 08.01.2019 08:00
- 6 Поставщик 1 08.01.2019 20:00 09.01.2019 08:00

Пояснение: для записей с 01.01.2019 по 04.01.2019 берется расписание ДВС

- 1 - Д - дневная смена далее следует выходной В - запись о выходном дне не попадает в таблицу
- 2 - С - суточная смена
- 3 - расписание закончилось, поэтому оно циклично начинается с начала (Д - дневная смена)
- 4 - С 05.01.2019 начинает действовать новое расписание - НВС берется Н - ночная смена далее следует выходной В - запись о выходном дне не попадает в таблицу
- 5 - С - суточная смена
- 6 - Снова Н - ночная смена

In [311]:
import pandas as pd
from datetime import datetime, timedelta

In [312]:
def my_sql(df_1, df_2):
    """
    Функция для заполнения таблицы T_CONTRACTOR_WORK_DAY.
    Принимает на вход таблицы T_CONTRACTOR_SHERULER и T_CONTRACTOR_WORK_DAY и возвращает заполненную таблицу T_CONTRACTOR_WORK_DAY.
    """
    
    count = 0
    for i in range(len(df_1)):
        curent_day = (df_1.iloc[i,3] - df_1.iloc[i,2]).days
        open = (df_1.iloc[i,2])
        while curent_day > 1:
            for j in range(len(df_1.iloc[i,1])):
                day = df_1.iloc[i,1]
                if day[j] == 'д':
                    T_CONTRACTOR_WORK_DAY.loc[count] = ([df_1.iloc[i,0], open + timedelta(hours=8), open + timedelta(hours=20)])
                    curent_day -= 1
                    open += timedelta(days=1)
                if day[j] == 'в':
                    curent_day -= 1
                    open += timedelta(days=1)
                if day[j] == 'с':
                    T_CONTRACTOR_WORK_DAY.loc[count] = ([df_1.iloc[i,0], open + timedelta(hours=8), open + timedelta(days=1) + timedelta(hours=8)])
                    curent_day -= 1
                    open += timedelta(days=1)
                if day[j] == 'н':
                    T_CONTRACTOR_WORK_DAY.loc[count] = ([df_1.iloc[i,0], open + timedelta(hours=20), open + timedelta(days=1) + timedelta(hours=8)])
                    curent_day -= 1
                    open += timedelta(days=1)
                count += 1
    df_2.reset_index(drop=True, inplace=True)
    return df_2

In [313]:
T_CONTRACTOR_SHERULER  = pd.read_csv('/content/drive/MyDrive/Elbrus/shedulers.csv', encoding=result['encoding'], sep =';')
T_CONTRACTOR_SHERULER

,ФИО,Расписание,Дата начала расписания,Дата окончания расписания
0,Поставщик 1,дддвсвнн,01.01.2019 0:00,10.01.2019 0:00
1,Поставщик 1,ннвннв,11.01.2019 0:00,15.01.2019 0:00
2,Поставщик 1,св,16.01.2019 0:00,20.01.2019 0:00
3,Поставщик 2,свсвсв,01.01.2019 0:00,07.01.2019 0:00
4,Поставщик 2,днвсв,08.01.2019 0:00,14.01.2019 0:00
5,Поставщик 2,ннддвсв,15.01.2019 0:00,31.12.9999 0:00
6,Поставщик 3,нвнвнв,01.01.2019 0:00,01.02.2019 0:00
7,Поставщик 3,двдвдвдв,02.02.2019 0:00,31.12.9999 0:00


In [315]:
# Преобразуем 'Дата начала расписания' и 'Дата окончания расписания' в формат datetime
T_CONTRACTOR_SHERULER['Дата начала расписания'] = T_CONTRACTOR_SHERULER['Дата начала расписания'].apply(lambda x: x[:11] + '00:00')
T_CONTRACTOR_SHERULER['Дата окончания расписания'] = T_CONTRACTOR_SHERULER['Дата окончания расписания'].apply(lambda x: x[:11] + '00:00')
T_CONTRACTOR_SHERULER.iloc[5,-1] = '31.12.2019 00:00'
T_CONTRACTOR_SHERULER.iloc[7,-1] = '31.12.2019 00:00'
T_CONTRACTOR_SHERULER['Дата начала расписания'] = pd.to_datetime(T_CONTRACTOR_SHERULER['Дата начала расписания'], format = "%d.%m.%Y %H:%M")
T_CONTRACTOR_SHERULER['Дата окончания расписания'] = pd.to_datetime(T_CONTRACTOR_SHERULER['Дата окончания расписания'], format = "%d.%m.%Y %H:%M")
T_CONTRACTOR_SHERULER

,ФИО,Расписание,Дата начала расписания,Дата окончания расписания
0,Поставщик 1,дддвсвнн,2019-01-01,2019-01-10
1,Поставщик 1,ннвннв,2019-01-11,2019-01-15
2,Поставщик 1,св,2019-01-16,2019-01-20
3,Поставщик 2,свсвсв,2019-01-01,2019-01-07
4,Поставщик 2,днвсв,2019-01-08,2019-01-14
5,Поставщик 2,ннддвсв,2019-01-15,2019-12-31
6,Поставщик 3,нвнвнв,2019-01-01,2019-02-01
7,Поставщик 3,двдвдвдв,2019-02-02,2019-12-31


In [316]:
# Создаем таблицу T_CONTRACTOR_WORK_DAY, ID - идентификатор записи является индексом
T_CONTRACTOR_WORK_DAY = pd.DataFrame(columns=['NAME', 'DATE_BEGIN', 'DATE_END'])

In [317]:
# Применяем функцию my_sql()
T_CONTRACTOR_WORK_DAY = my_sql(T_CONTRACTOR_SHERULER, T_CONTRACTOR_WORK_DAY)

# Задание 4
С помощью SQL запросов:

1. Сделать выборку содержащую сколько рабочих дней было у каждого поставщика
2. Сделать выборку поставщиков, у которыйх было больше 10 рабочих дней за январь 2019 года
3. Сделать выборку поставщиков, кто работал 14, 15 и 16 января 2019 года

1. Сделать выборку содержащую сколько рабочих дней было у каждого поставщика

In [318]:
T_CONTRACTOR_WORK_DAY.NAME.value_counts()

Поставщик 2    256
Поставщик 3    183
Поставщик 1     12
Name: NAME, dtype: int64

2. Сделать выборку поставщиков, у которыйх было больше 10 рабочих дней за январь 2019 года

In [319]:
T_CONTRACTOR_WORK_DAY[(T_CONTRACTOR_WORK_DAY['DATE_BEGIN'] >= '2019-01-01')&(T_CONTRACTOR_WORK_DAY['DATE_BEGIN'] < '2019-02-01')].NAME.value_counts()[T_CONTRACTOR_WORK_DAY[(T_CONTRACTOR_WORK_DAY['DATE_BEGIN'] >= '2019-01-01')&(T_CONTRACTOR_WORK_DAY['DATE_BEGIN'] < '2019-02-01')].NAME.value_counts()>10]

Поставщик 2    19
Поставщик 3    15
Поставщик 1    12
Name: NAME, dtype: int64

3. Сделать выборку поставщиков, кто работал 14, 15 и 16 января 2019 года

In [320]:
T_CONTRACTOR_WORK_DAY[(T_CONTRACTOR_WORK_DAY['DATE_BEGIN'] >= '2019-01-14')&(T_CONTRACTOR_WORK_DAY['DATE_BEGIN'] < '2019-01-17') | ((T_CONTRACTOR_WORK_DAY['DATE_END'] >= '2019-01-14')&(T_CONTRACTOR_WORK_DAY['DATE_END'] < '2019-01-17'))]

,NAME,DATE_BEGIN,DATE_END
8,Поставщик 1,2019-01-14 20:00:00,2019-01-15 08:00:00
9,Поставщик 1,2019-01-15 20:00:00,2019-01-16 08:00:00
10,Поставщик 1,2019-01-16 08:00:00,2019-01-17 08:00:00
18,Поставщик 2,2019-01-15 20:00:00,2019-01-16 08:00:00
19,Поставщик 2,2019-01-16 20:00:00,2019-01-17 08:00:00
274,Поставщик 3,2019-01-13 20:00:00,2019-01-14 08:00:00
275,Поставщик 3,2019-01-15 20:00:00,2019-01-16 08:00:00
